In [2]:
import matplotlib.pyplot as plt
import numpy as np
# Enable interactive plot

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib notebook

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, interact_manual
import numpy as np
from shapely.geometry import LineString
import matplotlib
import matplotlib.ticker as mticker
import cmath

ModuleNotFoundError: No module named 'shapely'

In [ ]:
def calc_speed(f,E):
    sigma = 0.33
    
    g  =9.81
    p_ice  =917
    v_p2 =E / (p_ice *  (1 - sigma**2))
    p_water = 1000
    c = np.arange(10,1500,1).astype(float)
    # c = 1000
    a = v_p2 * np.pi**2 * f**2 * H**3 * p_ice / (3*p_water)
    b =  (p_ice / p_water * H - g / (4*np.pi**2*f**2))
    d = np.zeros((1490,1))
    e = np.zeros((1490,1))
    velocity = np.zeros((270,1))
    
    for i in range(len(c)):
        d[i] = c[i]**5 /(2 * np.pi * f)
        e[i] = (a-b*c[i]**4 ) * abs(cmath.sqrt(1 - c[i]**2 / 1440**2))
        
    
    line_1 = LineString(np.column_stack((c,d)))
    line_2 = LineString(np.column_stack((c,e)))
    
    intersection = line_1.intersection(line_2)
    return intersection.xy[0]

In [ ]:
def animate(iframe):
    i=0
    for indx,freq in enumerate(frequency):
        #--------------------------------------------------------------------
        # ICE
        #--------------------------------------------------------------------
        omega = freq * np.pi * 2
        k_ice = omega / c_ice[indx]

        k_air = omega / c_air

        kx = k_ice
        ky = np.sqrt(k_air**2 - kx**2 + 0j)  #?
        
        

        dx = 0.01
        x = np.arange(0, xmax, dx)
        xx, yy = np.meshgrid(x_air, y_air)

        t = iframe * dt

        #y = np.exp(-1j * (omega*t - kx*x -ky*y))
        u_ice = np.real(np.exp(-1j * (omega*t - k_ice*x))) * (abs(ky)/k_air) * 10
        if i==0:
            u_tot = np.zeros((len(u_ice)))
            i=1
        u_tot +=u_ice
        #--------------------------------------------------------------------
        # ATHMOSPHERE
        #--------------------------------------------------------------------
        disp_u = np.real(np.exp(-1j * (omega*t - kx*xx -ky*yy))) * 10
        disp_v = np.real(np.exp(-1j * (np.pi/2. + omega*t - kx*xx -ky*yy))) * 10

        if i==1:
            disp_u_tot = np.zeros(disp_u.shape)
            disp_v_tot = np.zeros(disp_v.shape)
            i=2
        disp_u_tot +=disp_u
        disp_v_tot +=disp_v

        if kx < k_air: #Plane Wave
            disp_x = disp_u_tot * (kx/k_air)
            disp_y = disp_u_tot * (np.real(ky)/k_air)
        else: # Evanecent Wave
            disp_x =  disp_u_tot * (kx/k_air)
            disp_y =  -disp_v_tot * (abs(ky)/k_air)

        if i==2:
            disp_x_tot = np.zeros(disp_x.shape)
            disp_y_tot = np.zeros(disp_y.shape)
            i=3

        disp_x_tot +=disp_x
        disp_y_tot +=disp_y


    scat.set_data(((xx + disp_x_tot).flatten(), 
                       (yy + disp_y_tot).flatten()))

    line.set_data((x, u_tot))
    
    
        
    return 

In [ ]:
frequency = np.arange(65,75,1)
velocity = np.zeros((len(frequency),1))
E = [6.5e9]
H  =0.5

        

for E in(E):
    for i in range(len(frequency)):
        velocity[i] = calc_speed(frequency[i],E)
        
c_ice = velocity
#frequency =  [5,6,7,8,9]
#c_ice = [900,900,900,900,900]


In [ ]:
c_air = 330.
dt = 0.005
xmax = 200
ymax = 300
x_air = np.linspace(0, xmax, 80*2)
y_air = np.linspace(0, ymax, 40*2) 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
line, = ax.plot([], lw=3)     # A tuple unpacking to unpack the only plot
scat, = ax.plot([], '.', lw=0, ms=4) 
ax.set_xlim(0, xmax)
ax.set_ylim(-10, ymax)
ax.set_aspect('equal', 'box')
plt.tight_layout()


anim = FuncAnimation(fig, animate, frames=100, interval=20)
plt.show()

In [ ]:
frequency

In [ ]:
c_ice

In [ ]:
freq = 4        
omega = freq * np.pi * 2
k_ice = omega / c_ice
k_air = omega / c_air

kx = k_ice
ky = np.sqrt(k_air**2 - kx**2 + 0j)  #?


dx = 0.01
x = np.arange(0, xmax, dx)

xx, yy = np.meshgrid(x_air, y_air)

t = 2 * dt

#y = np.exp(-1j * (omega*t - kx*x -ky*y))
u_ice = np.real(np.exp(-1j * (omega*t - k_ice*x))) * (abs(ky)/k_air) * 10
    

line.set_data((x, u_tot))


In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))

ax.plot(x,u_ice)

In [ ]:
i=0
for indx,freq in enumerate(frequency):
        
    omega = freq * np.pi * 2
    k_ice = omega / c_ice[indx]

    k_air = omega / c_air

    kx = k_ice
    ky = np.sqrt(k_air**2 - kx**2 + 0j)  #?


    dx = 0.01
    x = np.arange(0, xmax, dx)

    xx, yy = np.meshgrid(x_air, y_air)

    t = 1 * dt

    #y = np.exp(-1j * (omega*t - kx*x -ky*y))
    u_ice = np.real(np.exp(-1j * (omega*t - k_ice*x))) * (abs(ky)/k_air) * 10
    if i==0:
        u_tot = np.zeros((len(u_ice)))
        i=1
    u_tot +=u_ice
    #--------------------------------------------------------------------
    disp_u = np.real(np.exp(-1j * (omega*t - kx*xx -ky*yy))) * 10
    disp_v = np.real(np.exp(-1j * (np.pi/2. + omega*t - kx*xx -ky*yy))) * 10
        
    if i==1:
        disp_u_tot = np.zeros(disp_u.shape)
        disp_v_tot = np.zeros(disp_v.shape)
        i=2
    disp_u_tot +=disp_u
    disp_v_tot +=disp_v
        
    if kx < k_air: #Plane Wave
        disp_x = disp_u_tot * (kx/k_air)
        disp_y = disp_u_tot * (np.real(ky)/k_air)
    else: # Evanecent Wave
        disp_x =  disp_u_tot * (kx/k_air)
        disp_y =  -disp_v_tot * (abs(ky)/k_air)
            
    if i==2:
        disp_x_tot = np.zeros(disp_x.shape)
        disp_y_tot = np.zeros(disp_y.shape)
        i=3
            
    disp_x_tot +=disp_x
    disp_y_tot +=disp_y

            
            

scat.set_data(((xx + disp_x_tot).flatten(), 
                   (yy + disp_y_tot).flatten()))
    
line.set_data((x, u_tot))

In [ ]:
disp_u_tot = np.zeros(disp_u.shape)


In [ ]:
disp_u_tot.shape